### The kinematics model of the Four-wheel steering vehicle
- 4WS bicycle model
![4WS bicycle model](pic/4WS_model.png)
    - Kinematic equation
$$
\begin{cases}
\dot x = v cos(\varphi + \beta)\\
\dot y = v sin(\varphi + \beta)\\
\dot \theta = \frac{v}{l} (tan\delta_f - tan\delta_r)cos\beta
\end{cases} \tag{1}
$$
 
$where \; \varphi \; is \; the \; yaw \; angle, \; \beta \; is \; the \; centroid \; side \; angle$

- In low speed scenes, the centroid angle can be approximated to zero($\beta \approx 0$), so (1) can be simplified to:
$$
\begin{cases}
\dot x = v cos(\varphi)\\
\dot y = v sin(\varphi)\\
\dot \theta = \frac{v}{l} (tan\delta_f - tan\delta_r)
\end{cases} \tag{2}
$$
> ***P. Hang, X. Chen, B. Zhang, P. Shi, T. Tang "Path Planning and Tracking Control for Collision
Avoidance of a 4WIS - 4WID Electric Vehicle." Automotive Engineering 2019(41).***
- The speed in the simplified model above is determined by the displacement per unit time, i.e. the motion step size. I set the step size to 0.1m in the algorithm.

### Search space
- I chose to sample the control input($[steer_f, \; steer_r, direction]$) instead of sampling the state($[x, \; y, \; \varphi]$).
    - $steer_f$: {-0.6, -0.48, -0.36, -0.24, -0.12, 0, 0.12, 0.24, 0.36, 0.48, 0.6}
    - $steer_r$: {-0.6, -0.48, -0.36, -0.24, -0.12, 0, 0.12, 0.24, 0.36, 0.48, 0.6}
    - direction: {1 , -1}
    - All possibilities from one node to the next: $11*11*2=242$(search space too large)
    - **Without the use of reeds sheep**, it took about **28 minutes** to find the path in reverse partking.
    ![reverse parking](pic/28min.png)
- In order to find the path faster, I reduced the search space.
    - $steer_f$: {-0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6}
    - $steer_r$: {-0.6, -0.3, 0, 0.3, 0.6}
    - direction: {1 , -1}
    - All possibilities from one node to the next: $7*5*2=70$
    - **With the use of reeds sheep**, it took about **2 minutes** to find the path in reverse partking.
    ![reverse parking](pic/2min.png)
- According to a simple comparison, the reeds sheep method can be used to speed up the results, but the disadvantage of reeds sheep is that the car always turns at the maximum corner, which may make it more difficult to find results in parallel parking.
    - improvement measures
        - Reeds sheep is executed only when the distance between the current point and the end point is less than a certain value, or only when the heuristic cost of the current point is less than a certain value.
        - Reduce the maximum curvature when in parallel parking

### Cost so far and cost to go
- the cost function for cost so far
    - backward penalty: cost += $w_b \times L$, where L is the motion step
    - switch direction penalty: cost += $w_{sw}, \; if \; d_i \neq d_{i-1}$
    - steer penalty: cost += $w_s \times (|steer_f| +|steer_r|)$
    - steer change penalty: cost += $w_{sc} \times |steer_f - steer_r|$
    - Adjusting the weight of each coefficient can speed up the search for results
    - TODO: further adjust the weight
    - It took about 1 minutes to find the path in reverse partking.
    ![reverse parking](pic/1min.png)
- the heuristic cost(cost to go)
    - Here I use the A-star algorithm to get the distance from the end point to each point on the map and take this as the heuristic cost
    - The resolution of the map largely affects the speed of the algorithm
    - In the case of  map resolution 1 m(0.3 -> 1), it took about **8 minutes** to find the path in parallel parking
    ![parallel parking](pic/8minparallel.png)

### collison check
1. use a two-step collision detection
    - The first step: a circle centered on the symmetric center of the car and diameter is the length of rectangular diagonal
    - The second step: Use the rectangle to detect, here the **safety distance($\epsilon$)** is added so that the rectangle is slightly larger than the car
2. use two different sized circles to approximate the car, like the picture below
![circular bounding regions](pic/circularbounding.png)
    - Faster than the first method

### Visualized results
- Dependency: 
    - python 3.7
    - numpy 1.16.1
    - scikit-learn 0.20.2
    - matplotlib 3.0.2
- In `AutonomousParking_demo/` you can get the results by executing `python3 main.py`, may need to wait about 3 minutes

- Reverse parking
![reverse parking(fix rear tire)](pic/Backwards_rear_fix.gif "Reverse")
    - The gif file is stored in `AutonomousParking_demo/results`
- Parallel parking
    - After modifying the algorithm, the result cannot be obtained for a long time. It is suspected that it is in an infinite loop. But in the reverse parking scenario, the algorithm can get results in a short time. So I need some time to debug and see what went wrong.